In [2]:
# 라이브러리
import requests, xmltodict, json
import pandas as pd

### 1) 서울특별시_정류소정보조회 서비스  
* 7_getStaionsByPosList 활용
* **input: tmX, tmY, radius => output: stId, stationName, arsId**
* https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15000303

In [3]:
# 사용자의 위치를 받아서 tmX, tmY 변수에 지정.
tmX = 126.9233021
tmY = 37.55639751
radius = 100 # 범위 (넓히면 여러 정류장 인식 됨.)

key = "AT98N5LWRAir0I67tVgrf6Vfnio9LCMcwusSbOjmdkEpSOGyobdyAq9cb41G6O4pgTp6Jcmpv8e87bplMNY7tQ%3D%3D"
url = f'http://ws.bus.go.kr/api/rest/stationinfo/getStationByPos?ServiceKey={key}&tmX={tmX}&tmY={tmY}&radius={radius}'
 
content = requests.get(url).content
dict = xmltodict.parse(content)

# 첫번째 정류장이라 설정 (음성으로 "현재 인식된 정류장은 A 정류장 입니다." 라고 전해주기)
jsonString = json.dumps(dict['ServiceResult']['msgBody']['itemList'][0]['stationNm'], ensure_ascii=False)
jsonObj = json.loads(jsonString)
# print(jsonObj)

# 향후 사용: 정류소 고유 번호, 정류소명 추출
target_stId = int(dict['ServiceResult']['msgBody']['itemList'][0]['stationId'])
target_stationName = str(dict['ServiceResult']['msgBody']['itemList'][0]['stationNm'])
target_arsId = str(dict['ServiceResult']['msgBody']['itemList'][0]['arsId'])
print(target_stId) # int
print(target_stationName)
print(target_arsId) # str

113000422
홍대입구역
14015


### 2) Excel Data를 통해 원하는 데이터 추출
* Excel Data: 1.busnumber_to_busRouteid , 2.stationName_to_ord 
* 다음 API 이용하는데 필요한 busRouteId, ord 추출하기 위함.
* 문제: 정류소 ord가 2020.9.2 기준이므로 이후 변경된 노선은 결과값 출력이 안됨

In [16]:
data1 = pd.read_csv('data/busnumber_to_busRouteid.csv', encoding='utf-8')
#data2 = pd.read_csv('data/stationName_to_ord.csv', encoding='utf-8')

# 음성인식을 통해 사용자가 5714번 탄다고 가정했음.
target_bus = '5714' # str 형태로 해야됨. -> 엑셀이 str 형태
target_busRouteId = data1[data1['busNumber'] == target_bus].iloc[0]['busRouteId']
print(target_busRouteId) # int

# ord 뽑기 = 엑셀 이용 방법
#target_ord = data2[(data2['stationName'] == target_stationName) & (data2['busNumber'] == target_bus) & (data2['stationNumber'] == int(target_arsId))].iloc[0]['ord']
#print(target_ord) # int

# ord 뽑기 = API 활용 방법
key = 'kNSQvU5WeosgTXwCx1mTthdz93%2BlLXHKA7ZtzbuNArBuUVVP4akW5xsfp6R5JYuMH106DwcuJRTqXJHI4q%2BNjA%3D%3D'
url = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={}&busRouteId={}'.format(key, target_busRouteId)

content = requests.get(url).content
dict = xmltodict.parse(content)

# target_arsId = arsId 넘버가 일치하는 버스의 seq(=ord) 구하기
alist = []
for i in range(0, len(dict['ServiceResult']['msgBody']['itemList'])):
    alist.append(dict['ServiceResult']['msgBody']['itemList'][i]['arsId'])   

# 인덱스 값이 곧 seq 값
target_ord = alist.index(target_arsId) + 1
print(target_ord)

100100288
42


### 3) 서울특별시_버스도착정보조회 서비스
* 2_getArrInfoByRouteList
* **input: stId, busRouteId, ord => output: arrmsg1, arrmsg2** + 이 외 필요한 것들 추가
* https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [17]:
#서울특별시_버스도착정보조회 서비스 中 2_getArrInfoByRouteList
key = 'kNSQvU5WeosgTXwCx1mTthdz93%2BlLXHKA7ZtzbuNArBuUVVP4akW5xsfp6R5JYuMH106DwcuJRTqXJHI4q%2BNjA%3D%3D'
url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRoute?ServiceKey={}&stId={}&busRouteId={}&ord={}'.format(key, target_stId, target_busRouteId, target_ord)

content = requests.get(url).content
dict = xmltodict.parse(content)

busstation = json.dumps(dict['ServiceResult']['msgBody']['itemList']['stNm'], ensure_ascii = False)
arrival = json.dumps(dict['ServiceResult']['msgBody']['itemList']['arrmsg1'], ensure_ascii = False)
arrival2 = json.dumps(dict['ServiceResult']['msgBody']['itemList']['arrmsg2'], ensure_ascii = False)
busplainnum = json.dumps(dict['ServiceResult']['msgBody']['itemList']['plainNo1'], ensure_ascii = False)
nextstation = json.dumps(dict['ServiceResult']['msgBody']['itemList']['stationNm1'], ensure_ascii = False)

jsonbusstation = json.loads(busstation)
jsonarrival = json.loads(arrival)
jsonarrival2 = json.loads(arrival2)
jsonbusplainnum = json.loads(busplainnum)
jsonnextstation = json.loads(nextstation)

print('정류소명: ' + jsonbusstation)
print('첫 번째 버스: ' + jsonarrival)
print('두 번째 버스: ' + jsonarrival2)
print('도착 예정 버스 차량 번호: ' + jsonbusplainnum)
print('다음 정류장: ' + jsonnextstation)

정류소명: 홍대입구역
첫 번째 버스: 2분후[2번째 전]
두 번째 버스: 9분56초후[5번째 전]
도착 예정 버스 차량 번호: 서울74사2919
다음 정류장: 서교동
